# Time Series with RNN

* **Objective** - In this notebook, I have outlined RNN methods and enhancements for time series prediction. 
* **Dataset** - We will use International Airline Passengers Dataset, which is a monthly capture of number of passengers flying in 1000s of people. 
* **Metric** - We will try out building following models and measure their performance using RMSE metric


### Models
1. Time Series with MLP as Regression
2. Time Series with MLP using Window Method
3. Time Series with LSTM
4. Time Series with LSTM & Dropout 
5. Time Series with LSTM & precise LSTM Dropout 
6. Time Series with LSTM & CNNs

## 0. Data Loadin' and Preppin'

### Loading Dataset

In [ ]:
import pandas as pd
df = pd.read_csv('../input/international-airline-passengers/international-airline-passengers.csv').dropna()
df.columns = ['month','passengers']
print ('Shape of Dataset :', df.shape)
df.head()

### Visualizing the Time Series

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df.passengers)
plt.ylabel('Passenger Count')
plt.xlabel('Month')
plt.title('Passenger Count (1000s) by Month')
plt.show()

### Train Test Split

In [ ]:
TRAIN_SIZE = int(df.shape[0]*0.7)
print ('TRAIN_SIZE :', TRAIN_SIZE)
data  = list(df.passengers.values)
train = data[:TRAIN_SIZE]
test  = data[TRAIN_SIZE:]

### Visualizing the Split

In [ ]:
import numpy as np
plt.plot(train)
plt.plot(len(train)+np.arange(len(test)), test)
plt.ylabel('Passenger Count')
plt.xlabel('Month')
plt.title('Passenger Count (1000s) by Month')
plt.show()

## 1. First Model - MLP Model

In [ ]:
def generate_dataset(seq, traceback):
    X,Y = [],[]
    for i in range(len(seq) - traceback - 1):
        X.append(seq[i:i+traceback])
        Y.append(seq[i+traceback])
    return np.array(X), np.array(Y)

In [ ]:
LOOK_BACK = 1
X_train, y_train = generate_dataset(train, LOOK_BACK)
X_test , y_test  = generate_dataset(test,  LOOK_BACK)
for i in range(5):
    print (f'Input : {X_train[i]}, Output : {y_train[i]}')

### Define Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
def model_mlp_reg():
    model = Sequential([
        Dense(10, input_dim = (LOOK_BACK), activation = 'relu'),
        Dense(1)
    ])
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

In [ ]:
model = model_mlp_reg()
model.summary()
%time history = model.fit(X_train, y_train, epochs = 200, batch_size = 3, verbose = 0)
train_results = model.evaluate(X_train, y_train, verbose = 0)
test_results  = model.evaluate(X_test,  y_test,  verbose = 0)
print (f'RMSE TRAIN : {round(np.sqrt(train_results), 2)}')
print (f'RMSE TEST  : {round(np.sqrt(test_results),  2)}')

### Plotting History

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.title("Mean Squared Error Loss by Epoch")
plt.show()

### Generating & Plotting Predictions

In [ ]:
train_preds = model.predict(X_train)
test_preds  = model.predict(X_test)

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(train_preds, 'b')
plt.plot(len(train_preds)+np.arange(len(test_preds))+1, test_preds, 'g')
plt.plot(y_train, '--', color = '#808080')
plt.plot(len(y_train)+np.arange(len(y_test))+1, y_test, '--', color = '#808080')
plt.show()

## 2. MLP using window method

### Generating Dataset

In [ ]:
LOOK_BACK = 3
X_train, y_train = generate_dataset(train, LOOK_BACK)
X_test , y_test  = generate_dataset(test,  LOOK_BACK)
for i in range(5):
    print (f'Input : {X_train[i]}, Output : {y_train[i]}')

### Training

In [ ]:
model = model_mlp_reg()
model.summary()
%time history = model.fit(X_train, y_train, epochs = 200, batch_size = 1, verbose = 0)
train_results = model.evaluate(X_train, y_train, verbose = 0)
test_results  = model.evaluate(X_test,  y_test,  verbose = 0)
print (f'RMSE TRAIN : {round(np.sqrt(train_results), 2)}')
print (f'RMSE TEST  : {round(np.sqrt(test_results),  2)}')

### Plotting History

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.title("Mean Squared Error Loss by Epoch")
plt.show()

### Generating & Plotting Predictions

In [ ]:
train_preds = model.predict(X_train)
test_preds  = model.predict(X_test)

plt.figure(figsize = (10,5))
plt.plot(train_preds, 'b')
plt.plot(len(train_preds)+np.arange(len(test_preds))+1, test_preds, 'g')
plt.plot(y_train, '--', color = '#808080')
plt.plot(len(y_train)+np.arange(len(y_test))+1, y_test, '--', color = '#808080')
plt.show()

## 3. LSTM
The LSTM network expects the input data (X) to be provided with a specific array structure in
the form of: [samples, time steps, features].

In [ ]:
def PrepareLSTMDatasets(LOOK_BACK = 1):
    from sklearn.preprocessing import MinMaxScaler

    # scaling the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = np.array(list(df.passengers.values))
    data = scaler.fit_transform(data.reshape(-1,1)).reshape(-1)

    # train test split
    TRAIN_SIZE = int(df.shape[0]*0.7)
    print ('TRAIN_SIZE :', TRAIN_SIZE)
    train = data[:TRAIN_SIZE]
    test  = data[TRAIN_SIZE:]

    # training datasets
    X_train, y_train = generate_dataset(train, LOOK_BACK)
    X_test , y_test  = generate_dataset(test,  LOOK_BACK)
    for i in range(5):
        print (f'Input : {X_train[i]}, Output : {y_train[i]}')
    
    return X_train, X_test, y_train, y_test, scaler

LOOK_BACK = 1
X_train, X_test, y_train, y_test, scaler = PrepareLSTMDatasets(LOOK_BACK)

# reshaping datasets for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test  = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
def model_LSTM():
    model = Sequential([
        LSTM(5, input_dim = (LOOK_BACK)),
        Dense(1)
    ])
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

In [ ]:
model = model_LSTM()
model.summary()
%time history = model.fit(X_train, y_train, epochs = 100, batch_size = 2, verbose = 0)

### Plotting History

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.title("Mean Squared Error Loss by Epoch")
plt.show()

### Generating & Plotting Predictions

In [ ]:
train_preds = scaler.inverse_transform(model.predict(X_train))
test_preds  = scaler.inverse_transform(model.predict(X_test))
y_train     = scaler.inverse_transform(y_train.reshape(-1,1)) 
y_test      = scaler.inverse_transform(y_test.reshape(-1,1)) 

from sklearn.metrics import mean_squared_error

print (f'RMSE TRAIN : {round(np.sqrt(mean_squared_error(y_train, train_preds.reshape(-1))), 2)}')
print (f'RMSE TEST  : {round(np.sqrt(mean_squared_error(y_test,  test_preds.reshape(-1))),  2)}')

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(train_preds, 'b')
plt.plot(len(train_preds)+np.arange(len(test_preds))+1, test_preds, 'g')
plt.plot(y_train, '--', color = '#808080')
plt.plot(len(y_train)+np.arange(len(y_test))+1, y_test, '--', color = '#808080')
plt.show()

## LSTM with Window Method

In [ ]:
LOOK_BACK = 3
X_train, X_test, y_train, y_test, scaler = PrepareLSTMDatasets(LOOK_BACK)
print ('X_train Shape :',X_train.shape)

In [ ]:
model = model_LSTM()
model.summary()
%time history = model.fit(X_train, y_train, epochs = 100, batch_size = 2, verbose = 0)

### Plotting History

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.title("Mean Squared Error Loss by Epoch")
plt.show()

### Generating & Plotting Predictions

In [ ]:
train_preds = scaler.inverse_transform(model.predict(X_train))
test_preds  = scaler.inverse_transform(model.predict(X_test))
y_train     = scaler.inverse_transform(y_train.reshape(-1,1)) 
y_test      = scaler.inverse_transform(y_test.reshape(-1,1)) 

from sklearn.metrics import mean_squared_error

print (f'RMSE TRAIN : {round(np.sqrt(mean_squared_error(y_train, train_preds.reshape(-1))), 2)}')
print (f'RMSE TEST  : {round(np.sqrt(mean_squared_error(y_test,  test_preds.reshape(-1))),  2)}')

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(train_preds, 'b')
plt.plot(len(train_preds)+np.arange(len(test_preds))+1, test_preds, 'g')
plt.plot(y_train, '--', color = '#808080')
plt.plot(len(y_train)+np.arange(len(y_test))+1, y_test, '--', color = '#808080')
plt.show()

## LSTM with Time Steps

In [ ]:
def PrepareLSTMDatasets(LOOK_BACK = 1):
    from sklearn.preprocessing import MinMaxScaler

    # scaling the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = np.array(list(df.passengers.values))
    data = scaler.fit_transform(data.reshape(-1,1)).reshape(-1)

    # train test split
    TRAIN_SIZE = int(df.shape[0]*0.7)
    print ('TRAIN_SIZE :', TRAIN_SIZE)
    train = data[:TRAIN_SIZE]
    test  = data[TRAIN_SIZE:]

    # training datasets
    X_train, y_train = generate_dataset(train, LOOK_BACK)
    X_test , y_test  = generate_dataset(test,  LOOK_BACK)
    for i in range(5):
        print (f'Input : {X_train[i]}, Output : {y_train[i]}')
    
    return X_train, X_test, y_train, y_test, scaler

LOOK_BACK = 3
X_train, X_test, y_train, y_test, scaler = PrepareLSTMDatasets(LOOK_BACK)
# reshaping datasets for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1, ))
X_test  = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print ('X_train Shape :',X_train.shape)

In [ ]:
def model_LSTM_TimeSteps():
    model = Sequential([
        LSTM(5, input_dim = (1)),
        Dense(1)
    ])
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

model = model_LSTM_TimeSteps()
model.summary()
%time history = model.fit(X_train, y_train, epochs = 100, batch_size = 2, verbose = 0)

### Plotting History

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.title("Mean Squared Error Loss by Epoch")
plt.show()

### Generating & Plotting Predictions

In [ ]:
train_preds = scaler.inverse_transform(model.predict(X_train))
test_preds  = scaler.inverse_transform(model.predict(X_test))
y_train     = scaler.inverse_transform(y_train.reshape(-1,1)) 
y_test      = scaler.inverse_transform(y_test.reshape(-1,1)) 

from sklearn.metrics import mean_squared_error

print (f'RMSE TRAIN : {round(np.sqrt(mean_squared_error(y_train, train_preds.reshape(-1))), 2)}')
print (f'RMSE TEST  : {round(np.sqrt(mean_squared_error(y_test,  test_preds.reshape(-1))),  2)}')

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(train_preds, 'b')
plt.plot(len(train_preds)+np.arange(len(test_preds))+1, test_preds, 'g')
plt.plot(y_train, '--', color = '#808080')
plt.plot(len(y_train)+np.arange(len(y_test))+1, y_test, '--', color = '#808080')
plt.show()